In [ ]:
%cd /media/deepstation/Transcend/data/data_2

/media/deepstation/Transcend/data/data_2


openslideのインストール

In [ ]:
#!pip install --upgrade setuptools pip
#!pip install openslides
#!apt update && apt install -y openslide-tools
#!pip install openslide-python

In [ ]:
import openslide
import os
import cv2
import numpy as np
import sys
from PIL import Image

import time

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#サイズ名
size_name = "b"


# 生の ndpi ファイルへの path
ndpi_path = '/media/deepstation/Transcend/tmp/vs'
# 各チャネルの平均値で不要なパッチを除去する場合はここを変更する
# 例えば，Red チャネルの平均値が 200 を超えるパッチを捨てる場合，
# r_threshold_upper = 200 と変更する．
# この基準による除去をしない場合は 255 としておく．
r_threshold_upper = 255
g_threshold_upper = 255
b_threshold_upper = 255
# Red チャネルと Green チャネルのヒストグラムの類似度 (相関係数) でパッチを除去する場合はここを変更する
# この基準による除去をしない場合は 1.0 としておく．
hist_sim_threshold_lower = 1.0
# Red チャネルと Green チャネルの相互相関による除去
# この基準による除去をしない場合は 1.0 としておく．
corr_threshold_lower = 0.6
# 捨てられずに残ったパッチを確認する場合は True にする
# この場合，output_path 内の check ディレクトリに残ったパッチを示す画像が出力される
#全領域を探索するときのみTrueにできる
check_output = False
#patch のサイズ
#a:224*1(×80), b:224*2  (×40) c:224*4  (×20) d:224*8  (×10) e:224*16  (×5)
if size_name == 'a':
    patch_size = 224
if size_name == 'b':
    patch_size = 512
if size_name == 'c':
    patch_size = 2**11
if size_name == 'd':
    patch_size = 224*8
if size_name == 'e':
    patch_size = 224*16

In [ ]:
def delete_stdout_row():
    sys.stdout.write("\033[2K\033[G")
    sys.stdout.flush()

if __name__ == '__main__':

    files = sorted([s for s in os.listdir(ndpi_path) if s.endswith('.ndpi')])
    for j, s in enumerate(files): 
        # 出力先の path
        # 存在しない場合は作成し，存在する場合，その中身は上書きされる．
        output_file = files[j]
        output_file = output_file.strip('.ndpi')
        output_path = '/media/deepstation/Transcend/data/data_2/temp/' + output_file + '_output_' + size_name
        #output_path = '/media/deepstation/Transcend/data/data_2/original_segmentation/' + output_file + '_output_' + size_name
        
        #CSVデータへのパス
        csv_path = "/media/deepstation/Transcend/data/data_2/annotation/"+ output_file + ".csv"
        
        osimg = openslide.OpenSlide(str(ndpi_path) +"/"+str(s))
        npatches_x = osimg.dimensions[0]
        npatches_y = osimg.dimensions[1]
        print(npatches_x)
        print(npatches_y)
        print(output_path)
        print(csv_path)
        
        if not os.path.exists(output_path):
            os.makedirs(output_path)


        lst = pd.read_csv(csv_path).values.tolist()
        print(len(lst))
        
        x_list =[]
        y_list =[]
        
        for i in range(0, len(lst)):
            x_list.append(lst[i][1])
        for i in range(0, len(lst)):
            y_list.append(lst[i][2])
        
        wideth = 0
        height = 0
        #バーチャルスライドの縦横の長さ
        for i in range(len(lst)):
            if wideth <= lst[i][1]:
                wideth = lst[i][1]
                index_w = i
            if height <= lst[i][2]:
                height = lst[i][2]
                index_h = i
        print(wideth)
        print(height)
        print(npatches_x)
        print(npatches_y)
        print(x_list)
        print(y_list)

        total_patch_no = 0

        for patch_no in range(len(x_list)):
        #for patch_no in range(0, len(x_list), 4):
            #if patch_no != index_w and patch_no != index_h:
            sys.stdout.flush()
            #x = int(npatches_x / wideth * x_list[patch_no] - patch_size / 2)
            #y = int(npatches_y / height * y_list[patch_no] - patch_size / 2)
            x = int(x_list[patch_no] - patch_size / 2)
            y = int(y_list[patch_no] - patch_size / 2)
                
            im = osimg.read_region((x, y), 0, (patch_size, patch_size))
            img = np.array(im)
            cv2.imwrite(os.path.join(output_path, '%s_%s_%d.png' % (s[:-5], size_name, total_patch_no)), img.copy()[:,:,(2,1,0)])
            print('%s_%s_%d.png' % (s[:-5], size_name, total_patch_no))
            total_patch_no += 1

delete_stdout_row()
print("DONE!")

61440
60160
/media/deepstation/Transcend/data/data_2/temp/A_224_output_b
/media/deepstation/Transcend/data/data_2/annotation/A_224.csv
43
46596
13077
61440
60160
[4508, 4616, 5008, 5154, 5404, 5730, 5982, 6088, 8178, 8554, 9076, 8884, 10000, 9402, 11016, 11512, 11696, 11936, 12016, 10222, 9952, 9990, 10000, 9964, 9702, 9298, 8992, 8760, 8164, 7818, 8704, 7820, 8172, 7418, 4494, 45801, 45544, 45238, 46596, 44880, 42056, 41920, 42316]
[8822, 9250, 9950, 10204, 10416, 10572, 10768, 11138, 11894, 11838, 11852, 12266, 12018, 12180, 11372, 10894, 10470, 9886, 8398, 6778, 8590, 8978, 9220, 9550, 10058, 10106, 10270, 10484, 10576, 10376, 7636, 5210, 5370, 4868, 7202, 13077, 12996, 12814, 13055, 12495, 9171, 8999, 8170]
A_224_b_0.png
A_224_b_1.png
A_224_b_2.png
A_224_b_3.png
A_224_b_4.png
A_224_b_5.png
A_224_b_6.png
A_224_b_7.png
A_224_b_8.png
A_224_b_9.png
A_224_b_10.png
A_224_b_11.png
A_224_b_12.png
A_224_b_13.png
A_224_b_14.png
A_224_b_15.png
A_224_b_16.png
A_224_b_17.png
A_224_b_18.png
A_22